In [ ]:
from datetime import datetime
import sys
sys.path.append('..')
import glob

import os
import pandas as pd 
import numpy as np

from config import *
from helpers.analysis import *

TABLES_DIR = os.path.join(paper, 'tables')
FIG_DIR = os.path.join(paper, 'figs')
for dir in [TABLES_DIR, FIG_DIR]:
    if not os.path.exists(dir):
        os.makedirs(dir)

maestro_subset_meta = pd.read_csv(EVAL_DATA_PATHS['maestro_subset']['meta'])

TIMESTAMP = datetime.now().strftime("%y%m%d_%H%M")
print(TIMESTAMP)

241125_1119


In [9]:
# Table 1: Evaluation set statistics
meta = pd.concat([maestro_subset_meta], ignore_index=True)

composers = meta.groupby(['composer'])['title'].nunique().index
num_pieces = meta.groupby(['composer'])['title'].nunique().values.astype(int)
num_performances = meta.groupby(
    ['composer'])['title'].count().values.astype(int)
duration_min = np.round(
    ((meta.groupby(['composer'])['duration_sec'].sum() / 60).values), 2)

dataset_table_columns = ['composer', 'pieces',
                         'performances', 'duration (min)']
dataset_table_arrays = [composers, num_pieces, num_performances, duration_min]
eval_set = pd.DataFrame({k: v for k, v in zip(dataset_table_columns, dataset_table_arrays)
                         })
# add total row
eval_set.loc['Total'] = eval_set.sum(numeric_only=True)
eval_set.fillna('Total', inplace=True)
eval_set
# prettify
eval_set['pieces'] = eval_set['pieces'].astype(int)
eval_set['performances'] = eval_set['performances'].astype(int)
# # save to latex
eval_set.to_latex(f'{TABLES_DIR}/tab1_eval_set_{TIMESTAMP}.tex', index=False,
                  escape=False, float_format="%.2f")

eval_set

,composer,pieces,performances,duration (min)
0,Bach,1,7,23.36
1,Beethoven,5,28,285.54
2,Chopin,4,15,150.28
3,Debussy,2,3,32.06
4,Glinka,1,2,10.35
5,Haydn,3,9,90.23
6,Liszt,3,12,58.98
7,Mozart,1,2,29.02
8,Rachmaninoff,2,3,11.87
9,Schubert,3,17,107.27


In [21]:
peamt = pd.read_csv(os.path.join(results, 'peamt.csv')
                    )  # no changes to peamt metric
peamt2 = pd.read_csv(
    '/share/hel/home/patricia/Research/tmp/tri24/results/peamt.csv')

print(peamt.equals(peamt2))

True


In [ ]:
# Table 2 : All metrics (IR, our musically informed, and PEAMT) overview per model for MAESTRO subset
# load all metrics

# frame_res = os.path.join(results, 'IR_frame_*.csv')
frame_res = pd.read_csv(glob.glob(os.path.join(results, 'IR_frame_*.csv'))[0])
note_offset_res = pd.read_csv(
    glob.glob(os.path.join(results, 'IR_note_offset_24*.csv'))[0])
note_offset_velocity_res = pd.read_csv(
    glob.glob(os.path.join(results, 'IR_note_offset_velocity_*.csv'))[0])

timing_res = pd.read_csv(glob.glob(os.path.join(results, 'musical_timing_*.csv'))[0])
articulation_res = pd.read_csv(glob.glob(os.path.join(results, 'musical_articulation_*.csv'))[0])
harmony_res = pd.read_csv(glob.glob(os.path.join(results, 'musical_harmony_*.csv'))[0])
dynamics_res = pd.read_csv(glob.glob(os.path.join(results, 'musical_dynamics_*.csv'))[0])

frame_metrics = ['f_f']
note_off_metrics = ['f_no']
note_off_vel_metrics = ['f_nov']
timing_metrics = ['melody_ioi_corr', 'acc_ioi_corr']
articulation_metrics = ['melody_kor_corr_64', 'bass_kor_corr_64', 'ratio_kor_corr_64']
harmony_metrics = ['cd_corr', 'cm_corr', 'ts_corr']
dynamics_metrics = ['dyn_corr']

peamt = pd.read_csv(os.path.join(results, 'peamt.csv')) # no changes to peamt metric

assert frame_res.shape[0] == note_offset_res.shape[0] == note_offset_velocity_res.shape[0] == timing_res.shape[0] == articulation_res.shape[0] == harmony_res.shape[0] == dynamics_res.shape[0] == peamt.shape[0]

# combine 
ALL_RES_MAESTRO = frame_res.copy()
ALL_RES_MAESTRO.drop(columns=['title', 'performer', 'p_f', 'r_f'], inplace=True)
ALL_RES_MAESTRO[note_off_metrics] = note_offset_res[note_off_metrics]
ALL_RES_MAESTRO[note_off_vel_metrics] = note_offset_velocity_res[note_off_vel_metrics]
ALL_RES_MAESTRO[timing_metrics] = timing_res[timing_metrics]
ALL_RES_MAESTRO[articulation_metrics] = articulation_res[articulation_metrics]
ALL_RES_MAESTRO[harmony_metrics] = harmony_res[harmony_metrics]
ALL_RES_MAESTRO[dynamics_metrics] = dynamics_res[dynamics_metrics]
ALL_RES_MAESTRO['peamt'] = peamt['peamt']
# filter to get only MAESTRO
ALL_RES_MAESTRO = ALL_RES_MAESTRO[ALL_RES_MAESTRO['recording'] == 'maestro']
ALL_RES_MAESTRO.drop(columns=['recording'], inplace=True)     
ALL_RES_MAESTRO.columns

all_metrics = ALL_RES_MAESTRO.columns.values[3:]
# sort by split
MAESTRO_RES_data_split = ALL_RES_MAESTRO.groupby(['model', 'split'])[all_metrics].mean()
MAESTRO_RES_data_split = sort_index(MAESTRO_RES_data_split, index_level='split')
MAESTRO_RES_data_split = sort_index(MAESTRO_RES_data_split, index_level='model')
MAESTRO_RES_data_split

# sort by model only
MAESTRO_RES_data = ALL_RES_MAESTRO.groupby(['model'])[all_metrics].mean()
MAESTRO_RES_data
# go from wide to long
MAESTRO_RES_table = MAESTRO_RES_data.transpose()
# sort and rename
MAESTRO_RES_table = MAESTRO_RES_table[['oaf', 'kong', 'T5']]
metrics_names_tables = {
    'f_f': 'Frame F1',
    'f_no': 'Note Offset F1',
    'f_nov': 'Note Offset Velocity F1',
    'melody_ioi_corr': 'Melody IOI',
    'acc_ioi_corr': 'Accompaniment IOI',
    'melody_kor_corr_64': 'Melody KOR',
    'bass_kor_corr_64': 'Bass KOR',
    'ratio_kor_corr_64': 'Ratio KOR',
    'cd_corr': 'Cloud Diameter',
    'cm_corr': 'Cloud Momentum',
    'ts_corr': 'Tensile Strain',
    'dyn_corr': 'Dynamics',
    'peamt': 'PEAMT'
}
MAESTRO_RES_table.index = MAESTRO_RES_table.index.map(metrics_names_tables)
# save to latex
MAESTRO_RES_table.to_latex(os.path.join(
    TABLES_DIR, f'tab2_metrics_overview_MAESTRO_subset_{TIMESTAMP}.tex'), escape=False, float_format="%.4f")

MAESTRO_RES_table

model,oaf,kong,T5
Frame F1,0.870516,0.913247,0.704477
Note Offset F1,0.772478,0.875069,0.804624
Note Offset Velocity F1,0.737181,0.860226,0.797790
Melody IOI,0.660790,0.538029,0.081922
Accompaniment IOI,0.484718,0.436163,0.306574
Melody KOR,0.520237,0.633022,0.345007
Bass KOR,0.297942,0.629828,0.318160
Ratio KOR,0.510017,0.615331,0.347967
Cloud Diameter,0.853846,0.830347,0.691246
Cloud Momentum,0.417686,0.395389,0.211309


In [23]:
# code from tri24 repo, ismir24-poster branch
peamt = pd.read_csv(
    '/share/hel/home/patricia/Research/tmp/mpteval/results/peamt_from_tri24.csv')

# combine
ALL_RES_MAESTRO = frame_res.copy()
ALL_RES_MAESTRO.drop(
    columns=['title', 'performer', 'p_f', 'r_f'], inplace=True)
ALL_RES_MAESTRO[note_off_metrics] = note_offset_res[note_off_metrics]
ALL_RES_MAESTRO[note_off_vel_metrics] = note_offset_velocity_res[note_off_vel_metrics]
ALL_RES_MAESTRO[timing_metrics] = timing_res[timing_metrics]
ALL_RES_MAESTRO[articulation_metrics] = articulation_res[articulation_metrics]
ALL_RES_MAESTRO[harmony_metrics] = harmony_res[harmony_metrics]
ALL_RES_MAESTRO[dynamics_metrics] = dynamics_res[dynamics_metrics]
ALL_RES_MAESTRO['peamt'] = peamt['peamt']
# filter to get only MAESTRO
ALL_RES_MAESTRO = ALL_RES_MAESTRO[ALL_RES_MAESTRO['recording'] == 'maestro']
ALL_RES_MAESTRO.drop(columns=['recording'], inplace=True)
ALL_RES_MAESTRO.columns

all_metrics = ALL_RES_MAESTRO.columns.values[3:]
# sort by split
MAESTRO_RES_data_split = ALL_RES_MAESTRO.groupby(['model', 'split'])[
    all_metrics].mean()
MAESTRO_RES_data_split = sort_index(
    MAESTRO_RES_data_split, index_level='split')
MAESTRO_RES_data_split = sort_index(
    MAESTRO_RES_data_split, index_level='model')
MAESTRO_RES_data_split

# sort by model only
MAESTRO_RES_data = ALL_RES_MAESTRO.groupby(['model'])[all_metrics].mean()
MAESTRO_RES_data
# go from wide to long
MAESTRO_RES_table = MAESTRO_RES_data.transpose()
# sort and rename
MAESTRO_RES_table = MAESTRO_RES_table[['oaf', 'kong', 'T5']]
metrics_names_tables = {
    'f_f': 'Frame F1',
    'f_no': 'Note Offset F1',
    'f_nov': 'Note Offset Velocity F1',
    'melody_ioi_corr': 'Melody IOI',
    'acc_ioi_corr': 'Accompaniment IOI',
    'melody_kor_corr_64': 'Melody KOR',
    'bass_kor_corr_64': 'Bass KOR',
    'ratio_kor_corr_64': 'Ratio KOR',
    'cd_corr': 'Cloud Diameter',
    'cm_corr': 'Cloud Momentum',
    'ts_corr': 'Tensile Strain',
    'dyn_corr': 'Dynamics',
    'peamt': 'PEAMT'
}
MAESTRO_RES_table.index = MAESTRO_RES_table.index.map(metrics_names_tables)

MAESTRO_RES_table

model,oaf,kong,T5
Frame F1,0.870516,0.913247,0.704477
Note Offset F1,0.772478,0.875069,0.804624
Note Offset Velocity F1,0.737181,0.860226,0.797790
Melody IOI,0.660790,0.538029,0.081922
Accompaniment IOI,0.484718,0.436163,0.306574
Melody KOR,0.520237,0.633022,0.345007
Bass KOR,0.297942,0.629828,0.318160
Ratio KOR,0.510017,0.615331,0.347967
Cloud Diameter,0.853846,0.830347,0.691246
Cloud Momentum,0.417686,0.395389,0.211309


In [19]:
# Table 3: Benchmark Results per model, split, and recording

# get mean frame fscore per model, split, and recording
frame_mean = frame_res.groupby(['model', 'split', 'recording'])[['f_f']].mean()
frame_mean = sort_index(frame_mean, 'model')
frame_mean = sort_index(frame_mean, 'split')
frame_mean = sort_index(frame_mean, 'recording')
frame_mean_values = frame_mean.values

# get mean note_offset fscore per model, split, and recording
note_offset_mean = note_offset_res.groupby(
    ['model', 'split', 'recording'])[['f_no']].mean()
note_offset_mean = sort_index(note_offset_mean, 'model')
note_offset_mean = sort_index(note_offset_mean, 'split')
note_offset_mean = sort_index(note_offset_mean, 'recording')
note_offset_mean_values = note_offset_mean.values

# get mean note_offset_velocity fscore per model, split, and recording
note_offset_velocity_mean = note_offset_velocity_res.groupby(
    ['model', 'split', 'recording'])[['f_nov']].mean()
note_offset_velocity_mean = sort_index(note_offset_velocity_mean, 'model')
note_offset_velocity_mean = sort_index(note_offset_velocity_mean, 'split')
note_offset_velocity_mean = sort_index(note_offset_velocity_mean, 'recording')
note_offset_velocity_mean_values = note_offset_velocity_mean.values

# combine them in one df
model_split_rec_res = frame_mean.index.to_frame(index=False)
model_split_rec_res['frame'] = frame_mean_values
model_split_rec_res['note_offset'] = note_offset_mean_values
model_split_rec_res['note_offset_velocity'] = note_offset_velocity_mean_values
model_split_rec_res

# pivot the table for the paper
model_split_rec_res_wide = model_split_rec_res.pivot(index=['split', 'recording'], columns='model', values=[
    'frame', 'note_offset', 'note_offset_velocity'])
# sort columns and index
columns_sorted = [
    ('frame',  'oaf'),
    ('frame', 'kong'),
    ('frame',   'T5'),
    ('note_offset',  'oaf'),
    ('note_offset', 'kong'),
    ('note_offset',   'T5'),
    ('note_offset_velocity',  'oaf'),
    ('note_offset_velocity', 'kong'),
    ('note_offset_velocity',   'T5')]
model_split_rec_res_wide = model_split_rec_res_wide[columns_sorted]

model_split_rec_res_wide = sort_index(model_split_rec_res_wide, 'split')
model_split_rec_res_wide = sort_index(model_split_rec_res_wide, 'recording')

# save to latex
model_split_rec_res_wide.to_latex(os.path.join(
    TABLES_DIR, f'tab3_benchmark_per_split_per_recording_{TIMESTAMP}.tex'), float_format="%.4f")

model_split_rec_res_wide

frame                     note_offset            \
model                       oaf      kong        T5         oaf      kong   
split      recording                                                        
train      maestro     0.880263  0.920137  0.725935    0.788960  0.890968   
           disklavier  0.818203  0.850310  0.615512    0.663042  0.758813   
validation maestro     0.840055  0.893145  0.654382    0.740682  0.863145   
           disklavier  0.769355  0.867455  0.609182    0.626064  0.823273   
test       maestro     0.852180  0.899495  0.654780    0.730630  0.824390   
           disklavier  0.804600  0.852540  0.604565    0.606260  0.723540   

                                note_offset_velocity                      
model                        T5                  oaf      kong        T5  
split      recording                                                      
train      maestro     0.822210             0.753036  0.876817  0.816676  
           disklavier  0.637131             0.606664  0.683608  0.575192  
validation maestro     0.806918             0.707736  0.848755  0.799245  
           disklavier  0.733464             0.580736  0.756827  0.670309  
test       maestro     0.740055             0.696295  0.806810  0.729000  
           disklavier  0.624815             0.554995  0.653415  0.556685